In [39]:
import pandas as pd 
import numpy as np 
import torch 
from sklearn.preprocessing import MinMaxScaler 
from sklearn.model_selection import train_test_split 
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer 

import torch 
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam
import xgboost

In [31]:
df = pd.read_csv("./archive/CellPhone_train.csv")

In [32]:
df.columns

Index(['battery_power', 'blue', 'clock_speed', 'dual_sim', 'fc', 'four_g',
       'int_memory', 'm_dep', 'mobile_wt', 'n_cores', 'pc', 'px_height',
       'px_width', 'ram', 'sc_h', 'sc_w', 'talk_time', 'three_g',
       'touch_screen', 'wifi', 'price_range'],
      dtype='object')

In [33]:
X, y = df.drop(columns=['price_range']), df[['price_range']]

In [34]:
float_cols = X.select_dtypes(include=[float]).columns.to_list()

In [35]:
int_cols = X.select_dtypes(include=[int]).columns.to_list()

In [36]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=123)

In [37]:
pipeline = ColumnTransformer(
    transformers = [('float_pipeline', MinMaxScaler(), float_cols)],
    remainder='passthrough')

In [38]:
pipeline.fit(X_train)

ColumnTransformer(remainder='passthrough',
                  transformers=[('float_pipeline', MinMaxScaler(),
                                 ['clock_speed', 'm_dep'])])

In [10]:
X_train = pipeline.transform(X_train)
X_test = pipeline.transform(X_test)

In [11]:
class PhoneDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y.values, dtype=torch.int32)

    def __getitem__(self, i):
        return self.X[i], self.y[i]

    def __len__(self):
        return len(self.X)
        

In [12]:
train_dataset = PhoneDataset(X_train, y_train)
test_dataset = PhoneDataset(X_test, y_test)

In [18]:
class NeuralNetwork(nn.Module):
    def __init__(self, input_num=20):
        super().__init__()
        
        self.net = nn.Sequential(
            nn.Linear(input_num, 15),
            nn.ReLU(),
            nn.Linear(15, 10),
            nn.ReLU(),
            nn.Linear(10, 4),
            nn.Softmax(dim=1)
        )

    def forward(self, X):
        X = self.net(X)
        return X

In [19]:
model = NeuralNetwork()

In [20]:
model

NeuralNetwork(
  (net): Sequential(
    (0): Linear(in_features=20, out_features=15, bias=True)
    (1): ReLU()
    (2): Linear(in_features=15, out_features=10, bias=True)
    (3): ReLU()
    (4): Linear(in_features=10, out_features=4, bias=True)
    (5): Softmax(dim=1)
  )
)

In [21]:
n_epochs = 20 
criterion = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=0.01)

BATCH_SIZE = 16
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = model.to(device)

In [29]:
train_acc = []
test_acc = []

for epoch in range(n_epochs):
    model.train()
    true_ans = 0
    for feat, target in train_loader:
        feat = feat.to(device)
        pred = model(feat)
        target = torch.eye(4)[target].squeeze().to(device)
        loss = criterion(pred, target)
        true_ans += (pred.argmax(dim=1) == target.argmax(dim=1)).sum().item()        
        #print(target.argmax(dim=1))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()  
    # running_train_acc = true_ans / len(train_dataset)
    # print(running_train_acc)
    # train_acc.append(running_train_acc) 

tensor(1.5562, device='cuda:0', grad_fn=<DivBackward1>)
tensor(1.2437, device='cuda:0', grad_fn=<DivBackward1>)
tensor(1.4937, device='cuda:0', grad_fn=<DivBackward1>)
tensor(1.6187, device='cuda:0', grad_fn=<DivBackward1>)
tensor(1.5562, device='cuda:0', grad_fn=<DivBackward1>)
tensor(1.4937, device='cuda:0', grad_fn=<DivBackward1>)
tensor(1.5562, device='cuda:0', grad_fn=<DivBackward1>)
tensor(1.3687, device='cuda:0', grad_fn=<DivBackward1>)
tensor(1.5562, device='cuda:0', grad_fn=<DivBackward1>)
tensor(1.6187, device='cuda:0', grad_fn=<DivBackward1>)
tensor(1.7437, device='cuda:0', grad_fn=<DivBackward1>)
tensor(1.4937, device='cuda:0', grad_fn=<DivBackward1>)
tensor(1.4937, device='cuda:0', grad_fn=<DivBackward1>)
tensor(1.4312, device='cuda:0', grad_fn=<DivBackward1>)
tensor(1.5562, device='cuda:0', grad_fn=<DivBackward1>)
tensor(1.4937, device='cuda:0', grad_fn=<DivBackward1>)
tensor(1.4312, device='cuda:0', grad_fn=<DivBackward1>)
tensor(1.3687, device='cuda:0', grad_fn=<DivBack

In [23]:
logits = torch.tensor([[0.1, 0.2, 0.3, 0.4, 0.5],
                       [0.5, 0.4, 0.3, 0.2, 0.1],
                       [0.2, 0.3, 0.4, 0.5, 0.6],
                       [0.4, 0.3, 0.2, 0.1, 0.0],
                       [0.5, 0.5, 0.5, 0.5, 0.5],
                       [0.1, 0.1, 0.1, 0.1, 0.1],
                       [0.3, 0.2, 0.1, 0.0, 0.5]])

# Integer targets
targets = torch.tensor([0, 3, 4, 4, 3, 0, 2])

# Initialize CrossEntropyLoss
criterion = nn.CrossEntropyLoss()

# Calculate the loss
loss = criterion(logits, targets)